In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import numpy as np
import torch

t_c = torch.tensor([0.5,14.0,15.0,28.0,11.0,8.0,3.0,-4.0,6.0,13.0,21.0])
t_u = torch.tensor([35.7,55.9,58.2,81.9,56.3,48.9,33.9,21.8,48.4,60.4,68.4])
print(t_u.shape)

t_c = t_c.unsqueeze(1)
t_u = t_u.unsqueeze(1)

n_samples = t_u.shape[0]
n_val = int(0.2*n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices=shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

print(train_indices,val_indices)

t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1*t_u_train
t_un_val = 0.1*t_u_val

torch.Size([11])
tensor([ 7,  9,  4,  3,  8,  5,  6, 10,  0]) tensor([2, 1])


In [4]:
import torch.nn as nn

seq_model = nn.Sequential(
    nn.Linear(1,13),
    nn.Tanh(),
    nn.Linear(13,1)
)

In [5]:
[param.shape for param in seq_model.parameters()]

[torch.Size([13, 1]), torch.Size([13]), torch.Size([1, 13]), torch.Size([1])]

In [6]:
for name,param in seq_model.named_parameters():
  print(name,param.shape)

0.weight torch.Size([13, 1])
0.bias torch.Size([13])
2.weight torch.Size([1, 13])
2.bias torch.Size([1])


In [7]:
from collections import OrderedDict

seq_model = nn.Sequential(OrderedDict([
    ('hidden_layer',nn.Linear(1,8)),
    ('hidden_activation',nn.Tanh()),
    ('output_linear',nn.Linear(8,1))
]))

In [8]:
seq_model

Sequential(
  (hidden_layer): Linear(in_features=1, out_features=8, bias=True)
  (hidden_activation): Tanh()
  (output_linear): Linear(in_features=8, out_features=1, bias=True)
)

In [9]:
for name,param in seq_model.named_parameters():
  print(name,param.shape)

hidden_layer.weight torch.Size([8, 1])
hidden_layer.bias torch.Size([8])
output_linear.weight torch.Size([1, 8])
output_linear.bias torch.Size([1])


In [11]:
import torch.optim as optim

optimizer = optim.SGD(seq_model.parameters(),lr=1e-3)

def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train,
                  t_u_val,t_c_train,t_c_val):
  for epoch in range(1,n_epochs+1):
    t_p_train = model(t_u_train)
    loss_train = loss_fn(t_p_train, t_c_train)

    t_p_val = model(t_u_val)
    loss_val = loss_fn(t_p_val,t_c_val)

    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()

    if epoch==1 or epoch%1000==0:
      print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
            f" Validation loss {loss_val.item():.4f}")


In [12]:

training_loop(
    n_epochs=5000,
    optimizer=optimizer,
    model=seq_model,
    loss_fn=nn.MSELoss(),
    t_u_train=t_un_train,
    t_u_val=t_un_val,
    t_c_train=t_c_train,
    t_c_val = t_c_val
)

Epoch 1, Training loss 200.3607, Validation loss 237.2892
Epoch 1000, Training loss 4.9626, Validation loss 2.7828
Epoch 2000, Training loss 1.5977, Validation loss 5.3840
Epoch 3000, Training loss 1.0651, Validation loss 7.1307
Epoch 4000, Training loss 0.9296, Validation loss 8.2911
Epoch 5000, Training loss 0.9016, Validation loss 8.7780


In [14]:
print('output',seq_model(t_un_val))
print('answer',t_c_val)
#print('hidden',seq_model.hidden_linear.weight.grad)

output tensor([[12.3387],
        [10.7637]], grad_fn=<AddmmBackward>)
answer tensor([[15.],
        [14.]])
